### File 07: GLM Regression with Tweedie

Our response data is very 0-heavy. We create a regression that accounts for this by using a Tweedie loss function. 

### Set up Spark session

We can specify more options in the SparkSession creator, but currently the options are at the default settings.

In [1]:
%%time
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col
from pyspark.sql.functions import log
from pyspark.ml.stat import Correlation

import pandas as pd
import numpy as np
import copy

spark = SparkSession.builder \
        .appName("project") \
        .getOrCreate()

sc = spark.sparkContext

CPU times: user 502 ms, sys: 416 ms, total: 918 ms
Wall time: 5.69 s


### Read in dataframes for train and test sets

This data should have been previously generated: we can find it in the `processed_data` folder.

In [2]:
%%time
trainDF = spark.read.parquet("./processed_data/train.parquet")
testDF = spark.read.parquet("./processed_data/test.parquet")
trainDF.show(5)

+---------+------------------+------------------+------------+--------------+------------------+------------------+----------------------------+---------------------------+----------------------------+----------------------------+------------------------+------------------------+-------------------------+------------------------+-----------+---------------+-----------+----------------------+------------------+------------------+-------------------------+---------------------+--------------------+
|  user_id|     T_total_spend|       total_spend|total_events|total_sessions|avg_session_length| sd_session_length|avg_interactions_per_session|sd_interactions_per_session|max_interactions_per_session|purchase_pct_of_total_events|view_pct_of_total_events|cart_pct_of_total_events|avg_purchases_per_session|sd_purchases_per_session|cart_events|purchase_events|view_events|sessions_with_purchase|sessions_with_cart|sessions_with_view|pct_sessions_end_purchase|pct_sessions_end_cart|       pca_purchas

In [3]:
trainDF = trainDF \
          .withColumn("total_spend_log", log(col("total_spend"))) \
          .withColumn("total_events_log", log(col("total_events"))) \
          .withColumn("purchase_events_log", log(col("purchase_events"))) \
          .withColumn("total_sessions_log", log(col("total_sessions"))) \
          .withColumn("T_total_spend_log", log(col("T_total_spend"))) \
          .withColumn("total_spend_pos", (col("total_spend"))) \
          .withColumn("total_events_pos", (col("total_events"))) \
          .withColumn("purchase_events_pos", (col("purchase_events"))) \
          .withColumn("total_sessions_pos", (col("total_sessions"))) \
          .withColumn("T_total_spend_pos", (col("T_total_spend")/100+1))

testDF = testDF \
          .withColumn("total_spend_log", log(col("total_spend"))) \
          .withColumn("total_events_log", log(col("total_events"))) \
          .withColumn("purchase_events_log", log(col("purchase_events"))) \
          .withColumn("total_sessions_log", log(col("total_sessions"))) \
          .withColumn("T_total_spend_log", log(col("T_total_spend"))) \
          .withColumn("total_spend_pos", (col("total_spend"))) \
          .withColumn("total_events_pos", (col("total_events"))) \
          .withColumn("purchase_events_pos", (col("purchase_events"))) \
          .withColumn("total_sessions_pos", (col("total_sessions"))) \
          .withColumn("T_total_spend_pos", (col("T_total_spend")/100+1))


### Set up Spark ML pipeline training for generalized linear regression

Here we decide which input columns should be used in order to create our training pipeline. To implement this step, we create the function `generatePipeline(inputCols, outputCol`). Then, we train the pipeline using this function.

In [4]:
%%time

inputCols = ["total_spend","total_events","purchase_events","total_sessions"]
# https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.GeneralizedLinearRegression.html
# on choosing variable values: https://www.rdocumentation.org/packages/statmod/versions/1.4.36/topics/tweedie
# LinkPower: Supported variables:
# variancePower: Supported values 0 and [1, inf)

def generateGLRPipeline(inputCols, outputCol):
    # Select input columns for generalized linear regression
    vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")
    
    # create glr instance & select output col
    glr = GeneralizedLinearRegression(featuresCol = "features", labelCol = outputCol, family = "tweedie")

    pipeline = Pipeline(stages=[vecAssembler, glr])
    return pipeline
    
pipeline = generateGLRPipeline(inputCols, "T_total_spend")
pipelineGLRModel = pipeline.fit(trainDF)

CPU times: user 18.6 ms, sys: 1.49 ms, total: 20.1 ms
Wall time: 1.66 s


In [5]:
def modelInfo(inputCols, pipelineGLRModel):
    # Create a zipped list containing the coefficients and the data
    modelCols = copy.deepcopy(inputCols)
    modelCoeffs = list(pipelineGLRModel.stages[-1].coefficients)
    modelCoeffs.insert(0,pipelineGLRModel.stages[-1].intercept)
    modelCols.insert(0,"intercept")
    modelZippedList = list(map(list, zip(modelCols, modelCoeffs)))

    # Create the pandas DataFrame
    modelDF = pd.DataFrame(modelZippedList, columns = ['Column name', 'Coefficient'])
    return modelDF

print("Model coefficients")
print(modelInfo(inputCols, pipelineGLRModel))


Model coefficients
       Column name  Coefficient
0        intercept  1494.143909
1      total_spend     4.279921
2     total_events   208.218305
3  purchase_events -2602.249440
4   total_sessions  -718.697638


In [6]:
# Calculate adjusted r2 (https://towardsdatascience.com/machine-learning-linear-regression-using-pyspark-9d5d5c772b42)
def adj_r2(r2, inputCols, testDF):
    n = testDF.count()
    p = len(inputCols)
    adjusted_r2 = 1-(((1-r2)*(n-1))/(n-p-1))
    return adjusted_r2

In [7]:
def getEvaluationMetrics(pipelineModel,outputCol,testDF,inputCols):
    predDF = pipelineModel.transform(testDF)
    predDF.select(outputCol, "prediction").show(10)

    regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol=outputCol,
    metricName="rmse")
    rmse = regressionEvaluator.evaluate(predDF)

    regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol=outputCol,
    metricName="r2")
    r2 = regressionEvaluator.evaluate(predDF)
    
    # Manually calculate Adjusted r2
    adjusted_r2 = adj_r2(r2, inputCols, testDF)
    
    return rmse, r2, adjusted_r2

evaluationMetrics = getEvaluationMetrics(pipelineGLRModel,"T_total_spend",testDF,inputCols)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")

+------------------+-------------------+
|     T_total_spend|         prediction|
+------------------+-------------------+
| 56475.99983215332|  164657.5830154838|
|1639.5600357055664|-303.32782546206204|
|1975.2000427246094| 1213.7693599057573|
|               0.0|-180.91606436870802|
|               0.0|  7309.178585129883|
| 61383.30047607422|  57233.91785304816|
|               0.0|-1848.6527619242759|
| 53453.60915565491|  95008.50069317133|
|               0.0|  1945.984141302463|
|               0.0|  6666.573428944875|
+------------------+-------------------+
only showing top 10 rows

RMSE is 35497.8
R^2 is 0.61312
Adjusted R^2 is 0.61290


#### While we want to do a tweedie model to test how a model built for zero-inflation works, it appears that this implementation is unable to create a mixture model using data with such a large tail as ours, even if we log-scale it. The working model above uses default parameters that are equivalent to a gaussian. However, we would like to examine variations on poisson-gamma distributions. As such, we will artificially limit our training dataset to only those with a response variable of < 100000, as otherwise we encounter an issue where the extremely large values cause a value to coerce to 0, breaking the model completely. 

In [8]:
trainDF2 = trainDF.filter(col("T_total_spend") <=1000)

##### Cross-evaluate tweedie variables (linkPower & varPower)

In [9]:
# For crossval to work, must define pipeline here
# Select input columns for generalized linear regression
vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")

# create glr instance & select output col
glr = GeneralizedLinearRegression(featuresCol = "features", labelCol = "T_total_spend_pos", family = "tweedie")

pipeline = Pipeline(stages=[vecAssembler, glr])

paramGrid = ParamGridBuilder() \
    .addGrid(glr.variancePower, [1]) \
    .addGrid(glr.linkPower, [2, 1.5]) \
    .build()

crossval = CrossValidator(estimator = pipeline,
                         estimatorParamMaps=paramGrid,
                         evaluator=RegressionEvaluator().setLabelCol("T_total_spend_pos"),
                         numFolds=4)
# Run cross-validation, and choose best set of parameters.
cvModel = crossval.fit(trainDF2)

Py4JJavaError: An error occurred while calling o373.fit.
: java.lang.AssertionError: assertion failed: Sum of weights cannot be zero.
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.ml.optim.WeightedLeastSquares$Aggregator.validate(WeightedLeastSquares.scala:426)
	at org.apache.spark.ml.optim.WeightedLeastSquares.fit(WeightedLeastSquares.scala:108)
	at org.apache.spark.ml.optim.IterativelyReweightedLeastSquares.fit(IterativelyReweightedLeastSquares.scala:91)
	at org.apache.spark.ml.regression.GeneralizedLinearRegression.$anonfun$train$1(GeneralizedLinearRegression.scala:431)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.regression.GeneralizedLinearRegression.train(GeneralizedLinearRegression.scala:379)
	at org.apache.spark.ml.regression.GeneralizedLinearRegression.train(G

In [ ]:
# Troubleshooting: https://stackoverflow.com/questions/49567921/pyspark-how-to-fit-a-glm-using-log-as-link-function-with-sum-of-weights-as-zero
# Sum of weights cannot be 0 issue
# Other suggestion is data is too long-tailed, try cutting it (treating more of spend as outlier)
#trainDF = trainDF.withColumn("T_total_spend_small", col("T_total_spend")/1000) # Per one person's advice, try making the response values smaller 

# code that produces "Sum of weights cannot be zero" is here: https://github.com/apache/spark/blob/master/mllib/src/main/scala/org/apache/spark/ml/optim/WeightedLeastSquares.scala


In [ ]:
# Testing param values to identify errors
#def generateGLRPipelineTEST(inputCols, outputCol, variancePower):
    # Select input columns for generalized linear regression
#    vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")
    
#    # create glr instance & select output col
#    glr = GeneralizedLinearRegression(featuresCol = "features", labelCol = outputCol, family = "tweedie", variancePower=variancePower)

#    pipeline = Pipeline(stages=[vecAssembler, glr])
#    return pipeline
#pipeline = generateGLRPipelineTEST(inputCols, "T_total_spend", 0)
#pipelineGLRModel = pipeline.fit(trainDF)


inputCols = ["total_spend_log", "total_events_log", "purchase_events_log", "total_sessions_log", "T_total_spend_pos"]


vecAssembler = VectorAssembler(inputCols=inputCols, outputCol="features")  
glr = GeneralizedLinearRegression(featuresCol = "features", labelCol = "T_total_spend_pos", family = "tweedie", linkPower=1.1)
pipeline = Pipeline(stages=[vecAssembler, glr])
pipelineGLRModel = pipeline.fit(trainDF2)

print("Model coefficients")
print(modelInfo(inputCols, pipelineGLRModel))

evaluationMetrics = getEvaluationMetrics(pipelineGLRModel,"T_total_spend_pos",testDF,inputCols)
print(f"RMSE is {evaluationMetrics[0]:.1f}")
print(f"R^2 is {evaluationMetrics[1]:.5f}")
print(f"Adjusted R^2 is {evaluationMetrics[2]:.5f}")